# 🗜️ Integer Discrete Flow (IDF) Training

Train a lossless image compression model using normalizing flows on ImageNet-1k crops.

In [ ]:
# Imports
import sys
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.amp import autocast, GradScaler
import numpy as np
from pathlib import Path
from datetime import datetime
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

# Local imports
from src.model import IntegerDiscreteFlow, create_idf_model
from src.dataset.crop_dataset import CropDataStream

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## Configuration

Set hyperparameters for training.

In [ ]:
# Configuration
config = {
    # Model architecture
    "in_channels": 3,
    "hidden_channels": 64,
    "num_levels": 3,
    "num_steps": 8,
    
    # Training
    "batch_size": 32,
    "learning_rate": 1e-4,
    "weight_decay": 1e-5,
    "grad_clip": 1.0,
    "total_steps": 50000,
    
    # Data
    "crop_size": 64,
    "dataset_name": "ILSVRC/imagenet-1k",
    
    # Logging & Checkpointing
    "log_interval": 100,
    "val_interval": 1000,
    "save_interval": 5000,
    "checkpoint_dir": "checkpoints",
    
    # Hardware
    "use_amp": True,  # Automatic mixed precision
}

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## Model Creation

Initialize the Integer Discrete Flow model.

In [ ]:
# Create model
model_config = {
    "in_channels": config["in_channels"],
    "hidden_channels": config["hidden_channels"],
    "num_levels": config["num_levels"],
    "num_steps": config["num_steps"],
}

model = create_idf_model(model_config).to(device)

# Count parameters
num_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Model config: {model_config}")
print(f"Total parameters: {num_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

## Data Loading

Create streaming dataloaders using `CropDataStream` for ImageNet-1k random crops.

In [ ]:
def collate_fn(batch):
    """Stack crops and scale to [0, 255] integer range."""
    crops = torch.stack(batch)
    # Convert from [0, 1] float to [0, 255] for the IDF model
    crops = (crops * 255.0).clamp(0, 255)
    return crops

# Create training dataset and dataloader
train_dataset = CropDataStream(
    split="train",
    crop_size=config["crop_size"],
    dataset_name=config["dataset_name"],
)

train_loader = DataLoader(
    train_dataset,
    batch_size=config["batch_size"],
    collate_fn=collate_fn,
    num_workers=0,  # Streaming dataset works best with 0 workers
    pin_memory=True if device.type == "cuda" else False,
)

# Create validation dataset and dataloader
val_dataset = CropDataStream(
    split="validation",
    crop_size=config["crop_size"],
    dataset_name=config["dataset_name"],
)

val_loader = DataLoader(
    val_dataset,
    batch_size=config["batch_size"],
    collate_fn=collate_fn,
    num_workers=0,
    pin_memory=True if device.type == "cuda" else False,
)

print("✓ Data loaders created")
print(f"  - Crop size: {config['crop_size']}x{config['crop_size']}")
print(f"  - Batch size: {config['batch_size']}")

## Training Setup

Configure optimizer, scheduler, and training utilities.

In [ ]:
# Optimizer
optimizer = optim.AdamW(
    model.parameters(),
    lr=config["learning_rate"],
    betas=(0.9, 0.999),
    weight_decay=config["weight_decay"],
)

# Learning rate scheduler (Cosine Annealing with Warm Restarts)
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
    optimizer,
    T_0=10000,  # Restart period
    T_mult=2,   # Period multiplier after each restart
)

# Mixed precision scaler
use_amp = config["use_amp"] and device.type == "cuda"
scaler = GradScaler("cuda") if use_amp else None

# Create checkpoint directory
checkpoint_dir = Path(config["checkpoint_dir"])
checkpoint_dir.mkdir(parents=True, exist_ok=True)

# Training state
global_step = 0
best_bpd = float("inf")
train_history = []

print(f"✓ Training setup complete")
print(f"  - Learning rate: {config['learning_rate']}")
print(f"  - Mixed precision: {use_amp}")

In [ ]:
def train_step(batch):
    """Perform a single training step with NaN detection."""
    global global_step
    
    model.train()
    batch = batch.to(device)
    optimizer.zero_grad()
    
    if use_amp:
        with autocast("cuda"):
            loss, bpd = model.compute_loss(batch)
        
        # NaN detection in loss/bpd
        if torch.isnan(loss) or torch.isnan(bpd):
            print(f"⚠️ NaN loss detected at step {global_step}")
            global_step += 1
            return {"loss": float("nan"), "bpd": float("nan"), "grad_norm": 0.0, 
                    "lr": scheduler.get_last_lr()[0], "nan_detected": True}
        
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), config["grad_clip"])
        
        # NaN gradient detection
        if torch.isnan(grad_norm):
            print(f"⚠️ NaN gradient at step {global_step}, skipping update")
            optimizer.zero_grad()
            scaler.update()  # Must call update() to reset scaler state
            global_step += 1
            return {"loss": loss.item(), "bpd": bpd.item(), "grad_norm": float("nan"),
                    "lr": scheduler.get_last_lr()[0], "nan_detected": True}
        
        scaler.step(optimizer)
        scaler.update()
    else:
        loss, bpd = model.compute_loss(batch)
        
        # NaN detection in loss/bpd
        if torch.isnan(loss) or torch.isnan(bpd):
            print(f"⚠️ NaN loss detected at step {global_step}")
            global_step += 1
            return {"loss": float("nan"), "bpd": float("nan"), "grad_norm": 0.0,
                    "lr": scheduler.get_last_lr()[0], "nan_detected": True}
        
        loss.backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), config["grad_clip"])
        
        # NaN gradient detection
        if torch.isnan(grad_norm):
            print(f"⚠️ NaN gradient at step {global_step}, skipping update")
            optimizer.zero_grad()
            global_step += 1
            return {"loss": loss.item(), "bpd": bpd.item(), "grad_norm": float("nan"),
                    "lr": scheduler.get_last_lr()[0], "nan_detected": True}
        
        optimizer.step()
    
    scheduler.step()
    global_step += 1
    
    return {
        "loss": loss.item(),
        "bpd": bpd.item(),
        "grad_norm": grad_norm.item() if isinstance(grad_norm, torch.Tensor) else grad_norm,
        "lr": scheduler.get_last_lr()[0],
        "nan_detected": False,
    }


@torch.no_grad()
def validate(val_loader, num_batches=50):
    """Run validation on a subset of validation data."""
    model.eval()
    total_bpd = 0.0
    count = 0
    
    for batch in val_loader:
        if count >= num_batches:
            break
        batch = batch.to(device)
        _, bpd = model.compute_loss(batch)
        total_bpd += bpd.item()
        count += 1
    
    return total_bpd / count if count > 0 else float("inf")


def save_checkpoint(name=None, is_best=False):
    """Save model checkpoint."""
    if name is None:
        name = f"checkpoint_step{global_step}.pt"
    
    checkpoint = {
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "scheduler_state_dict": scheduler.state_dict(),
        "global_step": global_step,
        "best_bpd": best_bpd,
        "train_history": train_history,
        "config": config,
    }
    
    if scaler is not None:
        checkpoint["scaler_state_dict"] = scaler.state_dict()
    
    torch.save(checkpoint, checkpoint_dir / name)
    
    if is_best:
        torch.save(checkpoint, checkpoint_dir / "best_model.pt")


def load_checkpoint(path):
    """Load model checkpoint."""
    global global_step, best_bpd, train_history
    
    checkpoint = torch.load(path, map_location=device)
    model.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    scheduler.load_state_dict(checkpoint["scheduler_state_dict"])
    global_step = checkpoint["global_step"]
    best_bpd = checkpoint["best_bpd"]
    train_history = checkpoint.get("train_history", [])
    
    if scaler is not None and "scaler_state_dict" in checkpoint:
        scaler.load_state_dict(checkpoint["scaler_state_dict"])
    
    print(f"✓ Loaded checkpoint from step {global_step}")

## Training Loop

Run the main training loop with logging, validation, and checkpointing.

In [ ]:
# Optional: Load from checkpoint to resume training
# load_checkpoint("checkpoints/checkpoint_step10000.pt")

print(f"Starting training from step {global_step}")
print(f"Target: {config['total_steps']} steps")
print(f"Device: {device}")
print("-" * 50)

# Training metrics accumulators
running_loss = 0.0
running_bpd = 0.0
running_count = 0

# NaN tracking
nan_count = 0
max_consecutive_nans = 10  # Stop if this many NaNs in a row

pbar = tqdm(total=config["total_steps"] - global_step, desc="Training")
train_iter = iter(train_loader)

while global_step < config["total_steps"]:
    # Get next batch
    try:
        batch = next(train_iter)
    except StopIteration:
        train_iter = iter(train_loader)
        batch = next(train_iter)
    
    # Training step
    metrics = train_step(batch)
    
    # Handle NaN detection
    if metrics.get("nan_detected", False):
        nan_count += 1
        if nan_count >= max_consecutive_nans:
            print(f"\n🛑 Training stopped: {nan_count} consecutive NaNs detected!")
            print("   Try: lower learning rate, disable AMP, or check input data")
            break
        pbar.update(1)
        continue
    else:
        nan_count = 0  # Reset counter on successful step
    
    running_loss += metrics["loss"]
    running_bpd += metrics["bpd"]
    running_count += 1
    
    # Logging
    if global_step % config["log_interval"] == 0:
        avg_loss = running_loss / running_count
        avg_bpd = running_bpd / running_count
        
        train_history.append({
            "step": global_step,
            "loss": avg_loss,
            "bpd": avg_bpd,
            "lr": metrics["lr"],
            "grad_norm": metrics["grad_norm"],
        })
        
        pbar.set_postfix({
            "loss": f"{avg_loss:.4f}",
            "bpd": f"{avg_bpd:.3f}",
            "lr": f"{metrics['lr']:.2e}",
        })
        
        running_loss = 0.0
        running_bpd = 0.0
        running_count = 0
    
    # Validation
    if global_step % config["val_interval"] == 0:
        val_bpd = validate(val_loader)
        print(f"\n[Step {global_step}] Val BPD: {val_bpd:.4f}")
        
        # Save best model
        if val_bpd < best_bpd:
            best_bpd = val_bpd
            save_checkpoint(is_best=True)
            print(f"  → New best model saved! (BPD: {best_bpd:.4f})")
    
    # Regular checkpointing
    if global_step % config["save_interval"] == 0:
        save_checkpoint()
    
    pbar.update(1)

pbar.close()

# Final save
save_checkpoint("final_model.pt")
print("\n" + "=" * 50)
print("Training complete!")
print(f"Best validation BPD: {best_bpd:.4f}")

## Training Visualization

Plot training curves to monitor progress.

In [ ]:
# Plot training curves
if len(train_history) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    steps = [h["step"] for h in train_history]
    bpds = [h["bpd"] for h in train_history]
    losses = [h["loss"] for h in train_history]
    lrs = [h["lr"] for h in train_history]
    grad_norms = [h["grad_norm"] for h in train_history]
    
    # BPD curve
    axes[0, 0].plot(steps, bpds, color="#e63946", linewidth=1.5)
    axes[0, 0].set_xlabel("Step")
    axes[0, 0].set_ylabel("Bits per Dimension (BPD)")
    axes[0, 0].set_title("Training BPD")
    axes[0, 0].grid(True, alpha=0.3)
    
    # Loss curve
    axes[0, 1].plot(steps, losses, color="#457b9d", linewidth=1.5)
    axes[0, 1].set_xlabel("Step")
    axes[0, 1].set_ylabel("Loss")
    axes[0, 1].set_title("Training Loss")
    axes[0, 1].grid(True, alpha=0.3)
    
    # Learning rate
    axes[1, 0].plot(steps, lrs, color="#2a9d8f", linewidth=1.5)
    axes[1, 0].set_xlabel("Step")
    axes[1, 0].set_ylabel("Learning Rate")
    axes[1, 0].set_title("Learning Rate Schedule")
    axes[1, 0].set_yscale("log")
    axes[1, 0].grid(True, alpha=0.3)
    
    # Gradient norm
    axes[1, 1].plot(steps, grad_norms, color="#f4a261", linewidth=1.5, alpha=0.8)
    axes[1, 1].set_xlabel("Step")
    axes[1, 1].set_ylabel("Gradient Norm")
    axes[1, 1].set_title("Gradient Norm")
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(checkpoint_dir / "training_curves.png", dpi=150, bbox_inches="tight")
    plt.show()
    
    print(f"\nFinal metrics:")
    print(f"  - Best BPD: {best_bpd:.4f}")
    print(f"  - Final BPD: {bpds[-1]:.4f}")
else:
    print("No training history to plot.")

## Test Compression

Verify the trained model can perfectly reconstruct images (lossless compression).

In [ ]:
# Test lossless reconstruction
model.eval()

# Get a test batch
test_iter = iter(val_loader)
test_batch = next(test_iter).to(device)

with torch.no_grad():
    # Compress
    latents, prior_params = model.compress(test_batch)
    
    # Decompress
    reconstructed = model.decompress(latents)
    
    # Check reconstruction quality
    reconstruction_error = (test_batch - reconstructed).abs().max().item()
    
print(f"Test batch shape: {test_batch.shape}")
print(f"Number of latent levels: {len(latents)}")
print(f"Max reconstruction error: {reconstruction_error:.6f}")

if reconstruction_error < 1.0:
    print("✓ Perfect lossless reconstruction achieved!")
else:
    print("⚠ Reconstruction has some error (expected for flow models with rounding)")

# Visualize original vs reconstructed
fig, axes = plt.subplots(2, 4, figsize=(12, 6))

for i in range(4):
    # Original
    orig_img = test_batch[i].cpu().numpy().transpose(1, 2, 0) / 255.0
    axes[0, i].imshow(np.clip(orig_img, 0, 1))
    axes[0, i].set_title(f"Original {i+1}")
    axes[0, i].axis("off")
    
    # Reconstructed
    recon_img = reconstructed[i].cpu().numpy().transpose(1, 2, 0) / 255.0
    axes[1, i].imshow(np.clip(recon_img, 0, 1))
    axes[1, i].set_title(f"Reconstructed {i+1}")
    axes[1, i].axis("off")

plt.suptitle("Lossless Compression Test", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()